In [4]:
import pandas as pd
import numpy as np

In [3]:
#Read the data using the Unnamed (probably id) as index
url = 'https://s3.amazonaws.com/drivendata/data/4/public/81e8f2de-9915-4934-b9ae-9705685c9d50.csv'
training = pd.read_csv(url, index_col='Unnamed: 0')

labels = ['Function', 'Object_Type', 'Operating_Status', 'Position_Type', 'Pre_K', 'Reporting', 
          'Sharing', 'Student_Type', 'Use']

numeric = ['FTE', 'Total']

categoric = [ 'Facility_or_Department', 'Function_Description', 
            'Fund_Description', 'Job_Title_Description', 'Location_Description', 
            'Object_Description', 'Position_Extra', 'Program_Description', 'SubFund_Description', 
            'Sub_Object_Description', 
            'Text_1', 'Text_2', 'Text_3', 'Text_4']

## Data cleaning

### Part 1: numeric data
First of all, the numeric columns `FTE` and `Total` must be cleaned. The outliers will be treated as `NaN` values and imputation will be applied to the missing data.

In [5]:
# Remove inconsistent data
training[training['FTE'] < 0] = np.nan
training[training['Total'] < 0] = np.nan

In [10]:
training['FTE'].describe()

count    125849.000000
mean          0.427436
std           0.573810
min           0.000000
25%           0.001078
50%           0.133337
75%           1.000000
max          46.800000
Name: FTE, dtype: float64

In [7]:
training[training['FTE'] < 0]

,Function,Use,Sharing,Reporting,Student_Type,Position_Type,Object_Type,Pre_K,Operating_Status,Object_Description,...,Sub_Object_Description,Location_Description,FTE,Function_Description,Facility_or_Department,Position_Extra,Total,Program_Description,Fund_Description,Text_1


In [11]:
training['Total'].describe()

count    3.518450e+05
mean     1.696423e+04
std      3.402292e+05
min      0.000000e+00
25%      1.384847e+02
50%      6.696800e+02
75%      4.999610e+03
max      1.297000e+08
Name: Total, dtype: float64

### Part 2: Categoric data

The strings must be normalized: symbols removed, all letters to lower case. Then, they can be transformed to numeric data.

In [13]:
for category in categoric:
    training[category] = training[category].str.lower()

In [16]:
#Check if the unique values decrease
training[categoric].nunique(dropna=False)

Facility_or_Department     176
Function_Description       655
Fund_Description           141
Job_Title_Description     3455
Location_Description       352
Object_Description         570
Position_Extra             580
Program_Description        418
SubFund_Description        267
Sub_Object_Description     159
Text_1                    1389
Text_2                     279
Text_3                      35
Text_4                     240
dtype: int64

In [18]:
#Convert columns
training[categoric] = training[categoric].apply(lambda x: x.astype('category'), axis = 0)

In [19]:
training[categoric].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400277 entries, 134338 to 415831
Data columns (total 14 columns):
Facility_or_Department    53186 non-null category
Function_Description      298676 non-null category
Fund_Description          179157 non-null category
Job_Title_Description     272317 non-null category
Location_Description      155504 non-null category
Object_Description        332143 non-null category
Position_Extra            244180 non-null category
Program_Description       280943 non-null category
SubFund_Description       263731 non-null category
Sub_Object_Description    70748 non-null category
Text_1                    251891 non-null category
Text_2                    87431 non-null category
Text_3                    108613 non-null category
Text_4                    53384 non-null category
dtypes: category(14)
memory usage: 13.8 MB


Before removing symbols, let's take a peek on some values and so we can detect the diferrences later


In [23]:
some_functions = training['Function_Description'].value_counts(normalize=True) > 0.05
some_functions.head(6)

non-project                        True
instruction                        True
unalloc budgets/schools            True
basic (fefp k-12)                 False
employee retirement               False
ela e-teaching sheltered eng      False
Name: Function_Description, dtype: bool

In [26]:
# Remove symbols
for category in categoric:
    training[category] = training[category].str.replace(r"[&/\)('\"-.,]","")

In [27]:
training['Function_Description'].value_counts(normalize=True).head(6)

nonproject                     0.191686
instruction                    0.128882
unalloc budgetsschools         0.055458
basic fefp k12                 0.044473
employee retirement            0.043723
ela eteaching sheltered eng    0.021093
Name: Function_Description, dtype: float64